In [1]:
import pickle
import numpy as np
import pandas as pd
from collections import defaultdict
from run_kernels import load_glove

/home/nlp/ravfogs/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
X,Y,X_dev, Y_dev, X_test, Y_test = load_glove(True)
q  = np.linalg.norm(X, axis=1)

In [3]:
import copy

def process(run_id, is_same=False):
    with open("interim/glove{}/kernel/preimage/eval/scores_with_multiple2.pickle".format(run_id), "rb") as f:
        scores = pickle.load(f)
    
    setting2scores = defaultdict(dict)

    for k,v in scores.items():
        params = k.split("_")[:]
        del params[1]
        #if "UniformMK-RBF" in k: continue
        
        params[1] = "$\gamma =  $" + str(params[1].split("=")[1])
        params[2] = "deg = "+str(params[2].split("=")[1])
        params[3] = "$\\alpha = $" + str(params[3].split("=")[1])
        params[0] = params[0].replace("kernel-type=", "")
        
        params_str = ", ".join(params[:-1]) if params[-1]!="same" else ", ".join(params[:-2])
        params_str = params_str.replace("d=", "\mathcal{l}=")
        #if "linear" in params_str and "adv-type=linear_same" in k: continue
    
        params_str = params_str.replace("None", "NA")
    
    
        adv = params[-1] if params[-1]!="same" else "-".join(params[-2:])
        #print(params_str, adv)
        if type(adv) == type([]): adv = "-".join(adv)
        
        
        adv = adv.replace("adv-type=","")
        if is_same:
                
            if params[-1] == "same" and params[0] in adv:
                adv = "same"
            elif params[-1] == "same":
                continue

        params_str = params_str.replace("kernel-type=","").replace(",unit-vecs=False", "").replace("l=1024", "").replace("..",".").replace("deg=", "deg = ").replace(".unit-vecs=False", "").replace(" unit-vecs=False", "")
        setting2scores[params_str][adv] = float(str(v)[:4]) #str(v)[:4] if v > 0.7 else "\textbf{"+format(str(v)[:4])+"}"
    
   
    cp = copy.deepcopy(setting2scores)
    if is_same:
        
        for k in cp.keys():
            for v in cp[k].keys():
                if not v in ["same"]:
                    del setting2scores[k][v]
    #    df = df[["linear", "same"]]
    else:
        for k in cp.keys():
            for v in cp[k].keys():
                if "same" in v:
                    del setting2scores[k][v]        
    return setting2scores

def mean_and_std(lst):
    
    means = defaultdict(dict)
    stds = defaultdict(dict)
    final = defaultdict(dict)
    
    for k in lst[0].keys():
        for v in lst[0][k].keys():
            mean = np.mean([d[k][v] for d in lst])
            std = np.std([d[k][v] for d in lst])
            means[k][v] = mean
            stds[k][v] = std
            final[k][v] = "{:.2f} $\pm$ {:.2f}".format(mean, std)
    df = pd.DataFrame(final).T
    return df
    
def mean_and_std2(lst):
    
    means = defaultdict(dict)
    stds = defaultdict(dict)
    final = []
    
    for k in lst[0].keys():
        for v in lst[0][k].keys():
            #if "linear" in k:
            mean = np.mean([d[k][v] for d in lst])
            std = np.std([d[k][v] for d in lst])
            means[k][v] = mean
            stds[k][v] = std
            kernel_type = k.split(",")[0]
            params = k.split(",")[1:]
            params = [p.replace(" ","").replace(",","").replace("$","").replace("NA","-").split("=")[1] for p in params]
            s = "{:.2f} $\pm$ {:.2f}".format(mean, std)
   
            final.append({"type": k, "Accuarcy": s, "adv": v})
            if "EasyMKL" in k:
                print(final[-1], "v:", v)
    df = pd.DataFrame(final)
    return df,final


def arrange_several_dataframes(lst):
    dfs = []
    
    data = []

    for d in lst:
        for k,v in d.items():
            d[k] = [v]
            kernel_type = k.split(",")[0]
            data.append({"type": kernel_type, "Accuracy": v["same"]})
    
    df = pd.DataFrame(data)
    kernels = df["type"].unique()
    final_data = []
    for k in kernels:
        mean = np.mean(df[df["type"]==k]["Accuracy"])
        std = np.std(df[df["type"]==k]["Accuracy"])
        final_data.append({"type": k, "Accuracy": "{:.2f} $\pm$ {:.2f}".format(mean, std)})
    return pd.DataFrame(final_data)

In [8]:
def average_over_kernel_type(setting2scores):
    kernel2scores = defaultdict(dict)
    kernels = set([k.split(",")[0] for k in setting2scores.keys()])
    for k in kernels:
        vals = [v["same"] for kk,v in setting2scores.items() if k in kk]
        keys = [kk for kk,v in setting2scores.items() if k in kk]
        for kk,vv in zip(keys,vals):
            kernel2scores[k][kk] = vv
    return kernel2scores

setting2scores1 = process(10, is_same=True)
setting2scores2 = process(11, is_same=True)
setting2scores3 = process(12, is_same=True)
setting2scores4 = process(13, is_same=True)
lst = [setting2scores1,setting2scores2,setting2scores3,setting2scores4]
df_same,final = mean_and_std2(lst) # evaluation when the adversary is the same kernel as the one being used in training
df_same_avg = arrange_several_dataframes(lst)

{'type': 'EasyMKL, $\\gamma =  $NA, deg = NA, $\\alpha = $NA', 'Accuarcy': '0.69 $\\pm$ 0.01', 'adv': 'same'} v: same


In [12]:
df_same_avg

,type,Accuracy
0,poly,0.59 $\pm$ 0.15
1,rbf,0.69 $\pm$ 0.16
2,laplace,0.75 $\pm$ 0.11
3,linear,0.54 $\pm$ 0.02
4,sigmoid,0.49 $\pm$ 0.00
5,EasyMKL,0.69 $\pm$ 0.01
6,UniformMK,0.49 $\pm$ 0.00


In [13]:
df_same

,type,Accuarcy,adv
0,"poly, $\gamma = $0.05, deg = 2, $\alpha = $0.8",0.49 $\pm$ 0.00,same
1,"poly, $\gamma = $0.05, deg = 2, $\alpha = $1",0.49 $\pm$ 0.00,same
2,"poly, $\gamma = $0.05, deg = 2, $\alpha = $1.2",0.49 $\pm$ 0.00,same
3,"poly, $\gamma = $0.05, deg = 3, $\alpha = $0.8",0.49 $\pm$ 0.00,same
4,"poly, $\gamma = $0.05, deg = 3, $\alpha = $1",0.49 $\pm$ 0.00,same
5,"poly, $\gamma = $0.05, deg = 3, $\alpha = $1.2",0.49 $\pm$ 0.00,same
6,"poly, $\gamma = $0.1, deg = 2, $\alpha = $0.8",0.49 $\pm$ 0.00,same
7,"poly, $\gamma = $0.1, deg = 2, $\alpha = $1",0.49 $\pm$ 0.00,same
8,"poly, $\gamma = $0.1, deg = 2, $\alpha = $1.2",0.49 $\pm$ 0.00,same
9,"poly, $\gamma = $0.1, deg = 3, $\alpha = $0.8",0.53 $\pm$ 0.01,same


In [26]:
setting2scores1 = process(10, is_same=False)
setting2scores2 = process(11, is_same=False)
setting2scores3 = process(12, is_same=False)
setting2scores4 = process(13, is_same=False)
lst = [setting2scores1,setting2scores2,setting2scores3,setting2scores4]
df_diff = mean_and_std(lst)  # evaluation when the adversary is different than same kernel as the one being used in training
df_diff


,UniformMK,EasyMKL,rbf,poly,laplace,sigmoid,linear,mlp
"poly, $\gamma = $0.05, deg = 2, $\alpha = $0.8",0.49 $\pm$ 0.00,0.98 $\pm$ 0.00,0.96 $\pm$ 0.00,0.98 $\pm$ 0.00,0.94 $\pm$ 0.01,0.49 $\pm$ 0.00,0.61 $\pm$ 0.06,0.97 $\pm$ 0.00
"poly, $\gamma = $0.05, deg = 2, $\alpha = $1",0.49 $\pm$ 0.00,0.98 $\pm$ 0.00,0.96 $\pm$ 0.01,0.98 $\pm$ 0.00,0.94 $\pm$ 0.00,0.49 $\pm$ 0.00,0.52 $\pm$ 0.01,0.97 $\pm$ 0.00
"poly, $\gamma = $0.05, deg = 2, $\alpha = $1.2",0.49 $\pm$ 0.00,0.98 $\pm$ 0.00,0.96 $\pm$ 0.00,0.98 $\pm$ 0.00,0.94 $\pm$ 0.00,0.49 $\pm$ 0.00,0.53 $\pm$ 0.02,0.97 $\pm$ 0.00
"poly, $\gamma = $0.05, deg = 3, $\alpha = $0.8",0.49 $\pm$ 0.00,0.98 $\pm$ 0.00,0.96 $\pm$ 0.01,0.98 $\pm$ 0.00,0.93 $\pm$ 0.01,0.49 $\pm$ 0.00,0.54 $\pm$ 0.03,0.97 $\pm$ 0.00
"poly, $\gamma = $0.05, deg = 3, $\alpha = $1",0.49 $\pm$ 0.00,0.98 $\pm$ 0.00,0.96 $\pm$ 0.00,0.98 $\pm$ 0.00,0.94 $\pm$ 0.01,0.49 $\pm$ 0.00,0.59 $\pm$ 0.05,0.97 $\pm$ 0.00
"poly, $\gamma = $0.05, deg = 3, $\alpha = $1.2",0.49 $\pm$ 0.00,0.98 $\pm$ 0.00,0.96 $\pm$ 0.00,0.98 $\pm$ 0.00,0.94 $\pm$ 0.00,0.49 $\pm$ 0.00,0.53 $\pm$ 0.01,0.97 $\pm$ 0.00
"poly, $\gamma = $0.1, deg = 2, $\alpha = $0.8",0.49 $\pm$ 0.00,0.98 $\pm$ 0.00,0.96 $\pm$ 0.00,0.98 $\pm$ 0.00,0.93 $\pm$ 0.01,0.49 $\pm$ 0.00,0.54 $\pm$ 0.04,0.97 $\pm$ 0.00
"poly, $\gamma = $0.1, deg = 2, $\alpha = $1",0.49 $\pm$ 0.00,0.98 $\pm$ 0.00,0.96 $\pm$ 0.00,0.98 $\pm$ 0.00,0.93 $\pm$ 0.01,0.49 $\pm$ 0.00,0.53 $\pm$ 0.02,0.97 $\pm$ 0.00
"poly, $\gamma = $0.1, deg = 2, $\alpha = $1.2",0.49 $\pm$ 0.00,0.98 $\pm$ 0.00,0.96 $\pm$ 0.00,0.98 $\pm$ 0.00,0.94 $\pm$ 0.00,0.49 $\pm$ 0.00,0.54 $\pm$ 0.01,0.97 $\pm$ 0.00
"poly, $\gamma = $0.1, deg = 3, $\alpha = $0.8",0.49 $\pm$ 0.00,0.98 $\pm$ 0.00,0.96 $\pm$ 0.00,0.98 $\pm$ 0.00,0.94 $\pm$ 0.00,0.49 $\pm$ 0.00,0.54 $\pm$ 0.00,0.97 $\pm$ 0.00


In [10]:
with pd.option_context("max_colwidth", 1000):

    df_same.to_csv("out-kernels/kernels-acc-preimage-same-cr.csv")
    df_diff.to_csv("out-kernels/kernels-acc-preimage-diff-cr.csv")
    df_same_avg.to_csv("out-kernels/kernels-acc-preimage-same-avg-cr.csv")